In [2]:
import pandas as pd
import numpy as np
from sklearn import linear_model, preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from scipy.spatial.distance import cdist
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.exceptions import ConvergenceWarning
import xgboost
from sklearn.decomposition import PCA
import warnings
import random
import time
import os
import sys
import argparse
import torch
from sklearn.cluster import KMeans
from sklearn_extra.cluster import KMedoids
from Bio import SeqIO

# Ignore FutureWarnings and ConvergenceWarnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning, module="sklearn.neural_network")
pd.options.mode.chained_assignment = None  # default='warn'

# set working directory to file location
os.chdir("/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks")

### Functions ported from grid_search

In [3]:
# Function to read in the data
def read_data(dataset_name, base_path, file_type, embeddings_type='both', experimental = False):
    # Construct the file paths
    if file_type == "csvs":
        labels_file = os.path.join(base_path, 'labels', dataset_name.split('_')[0] + '_labels.csv')
        hie_file = os.path.join(base_path, 'hie_temp', dataset_name.split('_')[0] + '.csv')
        embeddings_file = os.path.join(base_path, 'csvs', dataset_name + '.csv')
        # Read in mean embeddings across all rounds
        embeddings = pd.read_csv(embeddings_file, index_col=0)
    elif file_type == "pts":
        labels_file = os.path.join(base_path, 'labels', dataset_name.split('_')[-1] + '_labels.csv')
        hie_file = os.path.join(base_path, 'hie_temp', dataset_name.split('_')[-1] + '.csv')
        embeddings_file = os.path.join(base_path, 'pts', dataset_name + '.pt')
        # Read in pytorch tensor of embeddings
        embeddings = torch.load(embeddings_file)
        # Convert embeddings to a dataframe
        if embeddings_type == 'average':
            embeddings = {key: value['average'].numpy() for key, value in embeddings.items()}
        elif embeddings_type == 'mutated':
            embeddings = {key: value['mutated'].numpy() for key, value in embeddings.items()}
        elif embeddings_type == 'both':
            embeddings = {key: torch.cat((value['average'], value['mutated'])).numpy() for key, value in embeddings.items()}
        else:
            print("Invalid embeddings_type. Please choose 'average', 'mutated', or 'both'")
            return None, None

        # Convert embeddings dictionary to a dataframe
        embeddings = pd.DataFrame.from_dict(embeddings, orient='index')
    else:
        print("Invalid file type. Please choose either 'csvs' or 'pts'")
        return None, None

    # if not experimental
    if not experimental:
        # Read in labels
        labels = pd.read_csv(labels_file)

        # Read in hierarchy
        hie_data = pd.read_csv(hie_file)

        # Filter out rows where fitness is NaN
        labels = labels[labels['fitness'].notna()]

        # Filter out rows in embeddings where row names are not in labels variant column
        embeddings = embeddings[embeddings.index.isin(labels['variant'])]

        # Align labels by variant
        labels = labels.sort_values(by=['variant'])

        # Align embeddings by row name
        embeddings = embeddings.sort_index()

        # Confirm that labels and embeddings are aligned, reset index
        labels = labels.reset_index(drop=True)

        # Get the variants in labels and embeddings, convert to list
        label_variants = labels['variant'].tolist()
        embedding_variants = embeddings.index.tolist()

        # Check if embedding row names and label variants are identical
        if label_variants == embedding_variants:
            print('Embeddings and labels are aligned')

        # return embeddings and labels
        return embeddings, labels, hie_data

    else:
        return embeddings


# Active learning function for one iteration
def top_layer(iter_train, iter_test, embeddings_pd, labels_pd, measured_var, regression_type='ridge', top_n=None, final_round=10):

    # Get the variants in labels and embeddings, convert to list
    label_variants = labels_pd['variant'].tolist()
    embedding_variants = embeddings_pd.index.tolist()

    # Check if embedding row names and label variants are identical
    if label_variants == embedding_variants:
        print('Embeddings and labels are aligned')

    # reset the indices of embeddings_pd and labels_pd
    embeddings_pd = embeddings_pd.reset_index(drop=True)
    labels_pd = labels_pd.reset_index(drop=True)

    # save column 'iteration' in the labels dataframe
    iteration = labels_pd['iteration']

    # save labels
    labels = labels_pd

    # save mean embeddings as numpy array
    a = embeddings_pd

    # subset a, y to only include the rows where iteration = iter_train and iter_test
    idx_train = iteration[iteration.isin(iter_train)].index.to_numpy()
    idx_test = iteration[iteration.isin([iter_test])].index.to_numpy()

    # subset a to only include the rows where iteration = iter_train and iter_test
    X_train = a.loc[idx_train, :]
    X_test = a.loc[idx_test, :]

    y_train = labels[iteration.isin(iter_train)][measured_var]

    y_test = labels[iteration.isin([iter_test])][measured_var]

    # fit
    if regression_type == 'ridge':
        model = linear_model.RidgeCV()
    elif regression_type == 'lasso':
        model = linear_model.LassoCV(max_iter=100000,tol=1e-3)
    elif regression_type == 'elasticnet':
        model = linear_model.ElasticNetCV(max_iter=100000,tol=1e-3)
    elif regression_type == 'linear':
        model = linear_model.LinearRegression()
    elif regression_type == 'neuralnet':
        model = MLPRegressor(hidden_layer_sizes=(5), max_iter=1000, activation='relu', solver='adam', alpha=0.001,
                             batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5,
                             momentum=0.9, nesterovs_momentum=True, shuffle=True, random_state=1, tol=0.0001,
                             verbose=False, warm_start=False, early_stopping=False, validation_fraction=0.1, beta_1=0.9,
                             beta_2=0.999, epsilon=1e-08)
    elif regression_type == 'randomforest':
        model = RandomForestRegressor(n_estimators=100, criterion='friedman_mse', max_depth=None, min_samples_split=2,
                                      min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                                      max_leaf_nodes=None, min_impurity_decrease=0.0, bootstrap=True, oob_score=False,
                                      n_jobs=None, random_state=1, verbose=0, warm_start=False, ccp_alpha=0.0,
                                      max_samples=None)
    elif regression_type == 'gradientboosting':
        model = xgboost.XGBRegressor(objective='reg:squarederror', colsample_bytree=0.3, learning_rate=0.1,
                                     max_depth=5, alpha=10, n_estimators=10)

    model.fit(X_train, y_train)

    # make predictions on train data
    y_pred_train = model.predict(X_train)
    y_std_train = np.zeros(len(y_pred_train))
    # make predictions on test data
    # NOTE: can work on alternate 2-n round strategies here
    y_pred_test = model.predict(X_test)
    y_std_test = np.zeros(len(y_pred_test))

    # combine predicted and actual thermostability values with sequence IDs into a new dataframe
    df_train = pd.DataFrame({'variant': labels.variant[idx_train], 'y_pred': y_pred_train, 'y_actual': y_train})
    df_test = pd.DataFrame({'variant': labels.variant[idx_test], 'y_pred': y_pred_test, 'y_actual': y_test})
    
    # sort df_test by y_pred
    df_test = df_test.sort_values(by=['y_pred'], ascending=False)

    df_all = pd.concat([df_train, df_test])

    # sort df_all by y_pred
    df_all = df_all.sort_values(by=['y_pred'], ascending=False)

    return df_test, df_all

### New functions

In [4]:
def read_experimental_data(base_path, round_file_name, t7_sequence):
    file_path = base_path + '/rounds/' + round_file_name
    df = pd.read_excel(file_path)

    # Iterate through the 'Variant' column and update the values based on t7_sequence
    updated_variants = []
    for _, row in df.iterrows():
        variant = row['Variant']
        if variant == 'WT':
            updated_variants.append(variant)
        else:
            position = int(variant[:-1])
            wt_aa = t7_sequence[position - 1]
            updated_variant = wt_aa + variant
            updated_variants.append(updated_variant)
    
    df['updated_variant'] = updated_variants  # Add the updated variants to the DataFrame
    
    return df

def create_dataframes(df_list, expected_index):
    # First dataframe
    dfs = []  # List to store modified dataframes
    
    for i, df in enumerate(df_list, start=1):
        # Create a copy of the dataframe
        df_copy = df_list[i - 1].copy()
        # If the variant is WT, and i is equal to 1 assign iteration number 0
        if i == 1:
            df_copy.loc[df_copy['updated_variant'] == 'WT', 'iteration'] = 0
        else:
            df_copy = df_copy[df_copy['updated_variant'] != 'WT']
        df_copy.loc[df_copy['updated_variant'] != 'WT', 'iteration'] = i
        df_copy['iteration'] = df_copy['iteration'].astype(int)
        df_copy.rename(columns={'updated_variant': 'variant'}, inplace=True)  # Rename the column
    
        dfs.append(df_copy)

    df1 = pd.concat(dfs, ignore_index=True)
    df2 = pd.concat(dfs, ignore_index=True)

    df1 = df1[['variant', 'iteration']]
    df2 = df2[['variant', 'fitness', 'iteration']]

    expected_index_blank = [variant for variant in expected_index if variant not in df2['variant'].tolist()]
    # make a df_external that has a column 'variant' with all the variants in expected_index
    df_external = pd.DataFrame({'variant': expected_index_blank})
    df_external['fitness'] = np.nan  
    df_external['iteration'] = 1001 
    df2 = df2.append(df_external, ignore_index=True)
    # order df2 by expected_index
    df2 = df2.set_index('variant').reindex(expected_index, fill_value=np.nan).reset_index()
    # rename column 'index' to 'variant'
    df2 = df2.rename(columns={'index': 'variant'})
    
    return df1, df2

## T7 round 1

In [176]:
# import brenan data
dataset_name = 'esm2_15B_t7_pol'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
file_type = 'pts'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [177]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
WT,0.147168,-0.183875,0.031651,-0.052828,-0.141786,-0.013741,-0.061830,-0.028053,0.018626,0.025174,...,-0.110809,0.051659,0.059534,-0.157181,0.013038,0.020313,0.015764,0.064493,-0.008039,-0.108970
M1A,0.147994,-0.184054,0.035075,-0.053447,-0.140782,-0.016417,-0.063300,-0.028770,0.018573,0.022787,...,-0.111060,0.049681,0.060578,-0.156948,0.011523,0.018843,0.014856,0.063135,-0.005746,-0.106902
M1C,0.149553,-0.182680,0.033869,-0.054389,-0.140723,-0.015113,-0.064822,-0.028185,0.018523,0.027269,...,-0.111689,0.051024,0.059965,-0.157671,0.012664,0.019815,0.015817,0.063816,-0.007434,-0.108856
M1D,0.146734,-0.182717,0.034278,-0.052991,-0.140832,-0.015275,-0.063740,-0.028154,0.018495,0.025465,...,-0.111113,0.050927,0.060537,-0.156441,0.011660,0.019746,0.013886,0.063376,-0.006656,-0.107104
M1E,0.149375,-0.184232,0.034520,-0.054731,-0.140585,-0.016518,-0.064851,-0.028408,0.018998,0.025971,...,-0.110941,0.051635,0.058811,-0.157911,0.012537,0.020981,0.014563,0.062794,-0.007955,-0.106604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A883S,0.145864,-0.184623,0.031468,-0.051257,-0.142275,-0.012640,-0.061425,-0.027589,0.017693,0.025046,...,-0.110233,0.051852,0.058792,-0.157249,0.013417,0.018916,0.019343,0.065511,-0.009308,-0.109120
A883T,0.145933,-0.184290,0.031539,-0.051148,-0.141504,-0.012715,-0.062040,-0.028451,0.017504,0.024164,...,-0.110211,0.052196,0.058730,-0.158339,0.012717,0.018639,0.020539,0.065699,-0.010361,-0.109183
A883V,0.145253,-0.182491,0.031516,-0.052928,-0.142025,-0.013264,-0.060926,-0.027928,0.017077,0.024314,...,-0.111040,0.051419,0.060660,-0.157816,0.012737,0.018943,0.017333,0.065630,-0.008499,-0.109677
A883W,0.145614,-0.183985,0.031491,-0.051818,-0.142407,-0.012759,-0.059910,-0.027657,0.016999,0.023699,...,-0.110127,0.052603,0.059757,-0.156924,0.013284,0.018394,0.016497,0.064918,-0.009098,-0.108907


In [178]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
round_file_name = 'T7_Round1.xlsx'
t7_sequence = 'MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'
experimental_data = read_experimental_data(base_path, round_file_name, t7_sequence)
print(experimental_data)
df_list = [experimental_data]

  Variant   fitness updated_variant
0     12N  1.073846            S12N
1     25N  0.677227            A25N
2      WT  1.000000              WT
3     89R  0.740499            F89R
4    134T  1.074891           V134T
5    177L  1.042706           V177L
6    225E  1.075861           G225E
7    241W  0.938351           S241W
8    273H  0.785147           V273H


In [179]:
iterations_one, labels_one = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [180]:
df_all
df_all.to_csv('t7/round1_all_new.csv', index=False)

In [181]:
df_test

,variant,y_pred,y_actual
4266,G225M,0.994051,NaN
773,H41Q,0.992839,NaN
4019,V212L,0.989174,NaN
2539,V134N,0.987659,NaN
3334,H176L,0.987584,NaN
...,...,...,...
458,A25D,0.826342,NaN
385,F21G,0.824527,NaN
516,Y28D,0.821791,NaN
529,Y28S,0.816991,NaN


In [182]:
# write the dataframe to a csv file
df_test.to_csv('t7/round1_predictions_new.csv', index=False)

## T7 round 2

In [206]:
# import brenan data
dataset_name = 'esm2_15B_t7_pol'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
file_type = 'pts'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [207]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
WT,0.147168,-0.183875,0.031651,-0.052828,-0.141786,-0.013741,-0.061830,-0.028053,0.018626,0.025174,...,-0.110809,0.051659,0.059534,-0.157181,0.013038,0.020313,0.015764,0.064493,-0.008039,-0.108970
M1A,0.147994,-0.184054,0.035075,-0.053447,-0.140782,-0.016417,-0.063300,-0.028770,0.018573,0.022787,...,-0.111060,0.049681,0.060578,-0.156948,0.011523,0.018843,0.014856,0.063135,-0.005746,-0.106902
M1C,0.149553,-0.182680,0.033869,-0.054389,-0.140723,-0.015113,-0.064822,-0.028185,0.018523,0.027269,...,-0.111689,0.051024,0.059965,-0.157671,0.012664,0.019815,0.015817,0.063816,-0.007434,-0.108856
M1D,0.146734,-0.182717,0.034278,-0.052991,-0.140832,-0.015275,-0.063740,-0.028154,0.018495,0.025465,...,-0.111113,0.050927,0.060537,-0.156441,0.011660,0.019746,0.013886,0.063376,-0.006656,-0.107104
M1E,0.149375,-0.184232,0.034520,-0.054731,-0.140585,-0.016518,-0.064851,-0.028408,0.018998,0.025971,...,-0.110941,0.051635,0.058811,-0.157911,0.012537,0.020981,0.014563,0.062794,-0.007955,-0.106604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A883S,0.145864,-0.184623,0.031468,-0.051257,-0.142275,-0.012640,-0.061425,-0.027589,0.017693,0.025046,...,-0.110233,0.051852,0.058792,-0.157249,0.013417,0.018916,0.019343,0.065511,-0.009308,-0.109120
A883T,0.145933,-0.184290,0.031539,-0.051148,-0.141504,-0.012715,-0.062040,-0.028451,0.017504,0.024164,...,-0.110211,0.052196,0.058730,-0.158339,0.012717,0.018639,0.020539,0.065699,-0.010361,-0.109183
A883V,0.145253,-0.182491,0.031516,-0.052928,-0.142025,-0.013264,-0.060926,-0.027928,0.017077,0.024314,...,-0.111040,0.051419,0.060660,-0.157816,0.012737,0.018943,0.017333,0.065630,-0.008499,-0.109677
A883W,0.145614,-0.183985,0.031491,-0.051818,-0.142407,-0.012759,-0.059910,-0.027657,0.016999,0.023699,...,-0.110127,0.052603,0.059757,-0.156924,0.013284,0.018394,0.016497,0.064918,-0.009098,-0.108907


In [208]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
round_file_name_1 = 'T7_Round1.xlsx'
round_file_name_2 = 'T7_Round2.xlsx'
t7_sequence = 'MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, t7_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, t7_sequence)
print(experimental_data_1)
print(experimental_data_2)
df_list = [experimental_data_1, experimental_data_2]

  Variant   fitness updated_variant
0     12N  1.073846            S12N
1     25N  0.677227            A25N
2      WT  1.000000              WT
3     89R  0.740499            F89R
4    134T  1.074891           V134T
5    177L  1.042706           V177L
6    225E  1.075861           G225E
7    241W  0.938351           S241W
8    273H  0.785147           V273H
   Variant   fitness updated_variant
0     249C  1.055202           E249C
1     279S  0.999604           T279S
2     281L  0.589696           I281L
3     229I  0.478365           L229I
4     735S  1.840046           V735S
5     152N  1.908253           G152N
6     822S  1.678626           A822S
7     531T  1.096505           S531T
8     256P  0.600498           T256P
9     469Q  1.319594           G469Q
10    668E  1.279096           T668E
11      WT  1.000000              WT


In [209]:
iterations_two, labels_two = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_two
embeddings_pd = embeddings
labels_pd = labels_two
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [210]:
df_all
df_all.to_csv('t7/round2_all_new.csv', index=False)

In [211]:
df_test

,variant,y_pred,y_actual
2876,G152I,1.406724,NaN
2882,G152Q,1.364646,NaN
13958,V735N,1.323278,NaN
15031,R792C,1.320562,NaN
7763,A409N,1.320432,NaN
...,...,...,...
4814,I254H,0.828598,NaN
11151,I587V,0.827276,NaN
6750,E356G,0.824448,NaN
4561,S241A,0.822014,NaN


In [212]:
df_test.to_csv('t7/round2_predictions_new.csv', index=False)

## T7 round 3

In [239]:
# import brenan data
dataset_name = 'esm2_15B_t7_pol'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
file_type = 'pts'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [ ]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
WT,0.147168,-0.183875,0.031651,-0.052828,-0.141786,-0.013741,-0.061830,-0.028053,0.018626,0.025174,...,-0.110809,0.051659,0.059534,-0.157181,0.013038,0.020313,0.015764,0.064493,-0.008039,-0.108970
M1A,0.147994,-0.184054,0.035075,-0.053447,-0.140782,-0.016417,-0.063300,-0.028770,0.018573,0.022787,...,-0.111060,0.049681,0.060578,-0.156948,0.011523,0.018843,0.014856,0.063135,-0.005746,-0.106902
M1C,0.149553,-0.182680,0.033869,-0.054389,-0.140723,-0.015113,-0.064822,-0.028185,0.018523,0.027269,...,-0.111689,0.051024,0.059965,-0.157671,0.012664,0.019815,0.015817,0.063816,-0.007434,-0.108856
M1D,0.146734,-0.182717,0.034278,-0.052991,-0.140832,-0.015275,-0.063740,-0.028154,0.018495,0.025465,...,-0.111113,0.050927,0.060537,-0.156441,0.011660,0.019746,0.013886,0.063376,-0.006656,-0.107104
M1E,0.149375,-0.184232,0.034520,-0.054731,-0.140585,-0.016518,-0.064851,-0.028408,0.018998,0.025971,...,-0.110941,0.051635,0.058811,-0.157911,0.012537,0.020981,0.014563,0.062794,-0.007955,-0.106604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A883S,0.145864,-0.184623,0.031468,-0.051257,-0.142275,-0.012640,-0.061425,-0.027589,0.017693,0.025046,...,-0.110233,0.051852,0.058792,-0.157249,0.013417,0.018916,0.019343,0.065511,-0.009308,-0.109120
A883T,0.145933,-0.184290,0.031539,-0.051148,-0.141504,-0.012715,-0.062040,-0.028451,0.017504,0.024164,...,-0.110211,0.052196,0.058730,-0.158339,0.012717,0.018639,0.020539,0.065699,-0.010361,-0.109183
A883V,0.145253,-0.182491,0.031516,-0.052928,-0.142025,-0.013264,-0.060926,-0.027928,0.017077,0.024314,...,-0.111040,0.051419,0.060660,-0.157816,0.012737,0.018943,0.017333,0.065630,-0.008499,-0.109677
A883W,0.145614,-0.183985,0.031491,-0.051818,-0.142407,-0.012759,-0.059910,-0.027657,0.016999,0.023699,...,-0.110127,0.052603,0.059757,-0.156924,0.013284,0.018394,0.016497,0.064918,-0.009098,-0.108907


In [241]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
round_file_name_1 = 'T7_Round1.xlsx'
round_file_name_2 = 'T7_Round2.xlsx'
round_file_name_3 = 'T7_Round3.xlsx'
t7_sequence = 'MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, t7_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, t7_sequence)
experimental_data_3 = read_experimental_data(base_path, round_file_name_3, t7_sequence)
print(experimental_data_1)
print(experimental_data_2)
print(experimental_data_3)
df_list = [experimental_data_1, experimental_data_2, experimental_data_3]

  Variant   fitness updated_variant
0     12N  1.073846            S12N
1     25N  0.677227            A25N
2      WT  1.000000              WT
3     89R  0.740499            F89R
4    134T  1.074891           V134T
5    177L  1.042706           V177L
6    225E  1.075861           G225E
7    241W  0.938351           S241W
8    273H  0.785147           V273H
   Variant   fitness updated_variant
0     249C  1.055202           E249C
1     279S  0.999604           T279S
2     281L  0.589696           I281L
3     229I  0.478365           L229I
4     735S  1.840046           V735S
5     152N  1.908253           G152N
6     822S  1.678626           A822S
7     531T  1.096505           S531T
8     256P  0.600498           T256P
9     469Q  1.319594           G469Q
10    668E  1.279096           T668E
11      WT  1.000000              WT
   Variant   fitness updated_variant
0      10K  0.959845            D10K
1     643N  3.909999           E643N
2     370V  2.600643           N370V
3       5I 

In [242]:
iterations_three, labels_three = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_three
embeddings_pd = embeddings
labels_pd = labels_three
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [243]:
df_all
df_all.to_csv('t7/round3_all_new.csv', index=False)

In [244]:
df_test

,variant,y_pred,y_actual
12213,E643S,2.398941,NaN
12206,E643K,2.301938,NaN
12199,E643A,2.223764,NaN
12212,E643R,2.193382,NaN
12214,E643T,2.162055,NaN
...,...,...,...
14457,I761V,0.880633,NaN
4715,E249D,0.875294,NaN
6743,I355V,0.866922,NaN
11151,I587V,0.864906,NaN


In [245]:
df_test.to_csv('t7/round3_predictions_new.csv', index=False)

## T7 round 4

In [5]:
# import brenan data
dataset_name = 'esm2_15B_t7_pol'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
file_type = 'pts'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [6]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
WT,0.147168,-0.183875,0.031651,-0.052828,-0.141786,-0.013741,-0.061830,-0.028053,0.018626,0.025174,...,-0.110809,0.051659,0.059534,-0.157181,0.013038,0.020313,0.015764,0.064493,-0.008039,-0.108970
M1A,0.147994,-0.184054,0.035075,-0.053447,-0.140782,-0.016417,-0.063300,-0.028770,0.018573,0.022787,...,-0.111060,0.049681,0.060578,-0.156948,0.011523,0.018843,0.014856,0.063135,-0.005746,-0.106902
M1C,0.149553,-0.182680,0.033869,-0.054389,-0.140723,-0.015113,-0.064822,-0.028185,0.018523,0.027269,...,-0.111689,0.051024,0.059965,-0.157671,0.012664,0.019815,0.015817,0.063816,-0.007434,-0.108856
M1D,0.146734,-0.182717,0.034278,-0.052991,-0.140832,-0.015275,-0.063740,-0.028154,0.018495,0.025465,...,-0.111113,0.050927,0.060537,-0.156441,0.011660,0.019746,0.013886,0.063376,-0.006656,-0.107104
M1E,0.149375,-0.184232,0.034520,-0.054731,-0.140585,-0.016518,-0.064851,-0.028408,0.018998,0.025971,...,-0.110941,0.051635,0.058811,-0.157911,0.012537,0.020981,0.014563,0.062794,-0.007955,-0.106604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
A883S,0.145864,-0.184623,0.031468,-0.051257,-0.142275,-0.012640,-0.061425,-0.027589,0.017693,0.025046,...,-0.110233,0.051852,0.058792,-0.157249,0.013417,0.018916,0.019343,0.065511,-0.009308,-0.109120
A883T,0.145933,-0.184290,0.031539,-0.051148,-0.141504,-0.012715,-0.062040,-0.028451,0.017504,0.024164,...,-0.110211,0.052196,0.058730,-0.158339,0.012717,0.018639,0.020539,0.065699,-0.010361,-0.109183
A883V,0.145253,-0.182491,0.031516,-0.052928,-0.142025,-0.013264,-0.060926,-0.027928,0.017077,0.024314,...,-0.111040,0.051419,0.060660,-0.157816,0.012737,0.018943,0.017333,0.065630,-0.008499,-0.109677
A883W,0.145614,-0.183985,0.031491,-0.051818,-0.142407,-0.012759,-0.059910,-0.027657,0.016999,0.023699,...,-0.110127,0.052603,0.059757,-0.156924,0.013284,0.018394,0.016497,0.064918,-0.009098,-0.108907


In [9]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/t7/'
round_file_name_1 = 'T7_Round1.xlsx'
round_file_name_2 = 'T7_Round2.xlsx'
round_file_name_3 = 'T7_Round3.xlsx'
round_file_name_4 = 'T7_Round4.xlsx'
t7_sequence = 'MNTINIAKNDFSDIELAAIPFNTLADHYGERLAREQLALEHESYEMGEARFRKMFERQLKAGEVADNAAAKPLITTLLPKMIARINDWFEEVKAKRGKRPTAFQFLQEIKPEAVAYITIKTTLACLTSADNTTVQAVASAIGRAIEDEARFGRIRDLEAKHFKKNVEEQLNKRVGHVYKKAFMQVVEADMLSKGLLGGEAWSSWHKEDSIHVGVRCIEMLIESTGMVSLHRQNAGVVGQDSETIELAPEYAEAIATRAGALAGISPMFQPCVVPPKPWTGITGGGYWANGRRPLALVRTHSKKALMRYEDVYMPEVYKAINIAQNTAWKINKKVLAVANVITKWKHCPVEDIPAIEREELPMKPEDIDMNPEALTAWKRAAAAVYRKDKARKSRRISLEFMLEQANKFANHKAIWFPYNMDWRGRVYAVSMFNPQGNDMTKGLLTLAKGKPIGKEGYYWLKIHGANCAGVDKVPFPERIKFIEENHENIMACAKSPLENTWWAEQDSPFCFLAFCFEYAGVQHHGLSYNCSLPLAFDGSCSGIQHFSAMLRDEVGGRAVNLLPSETVQDIYGIVAKKVNEILQADAINGTDNEVVTVTDENTGEISEKVKLGTKALAGQWLAYGVTRSVTKRSVMTLAYGSKEFGFRQQVLEDTIQPAIDSGKGLMFTQPNQAAGYMAKLIWESVSVTVVAAVEAMNWLKSAAKLLAAEVKDKKTGEILRKRCAVHWVTPDGFPVWQEYKKPIQTRLNLMFLGQFRLQPTINTNKDSEIDAHKQESGIAPNFVHSQDGSHLRKTVVWAHEKYGIESFALIHDSFGTIPADAANLFKAVRETMVDTYESCDVLADFYDQFADQLHESQLDKMPALPAKGNLNLRDILESDFAFA'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, t7_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, t7_sequence)
experimental_data_3 = read_experimental_data(base_path, round_file_name_3, t7_sequence)
experimental_data_4 = read_experimental_data(base_path, round_file_name_4, t7_sequence)
print(experimental_data_1)
print(experimental_data_2)
print(experimental_data_3)
print(experimental_data_4)
df_list = [experimental_data_1, experimental_data_2, experimental_data_3, experimental_data_4]

  Variant   fitness updated_variant
0     12N  1.073846            S12N
1     25N  0.677227            A25N
2      WT  1.000000              WT
3     89R  0.740499            F89R
4    134T  1.074891           V134T
5    177L  1.042706           V177L
6    225E  1.075861           G225E
7    241W  0.938351           S241W
8    273H  0.785147           V273H
   Variant   fitness updated_variant
0     249C  1.055202           E249C
1     279S  0.999604           T279S
2     281L  0.589696           I281L
3     229I  0.478365           L229I
4     735S  1.840046           V735S
5     152N  1.908253           G152N
6     822S  1.678626           A822S
7     531T  1.096505           S531T
8     256P  0.600498           T256P
9     469Q  1.319594           G469Q
10    668E  1.279096           T668E
11      WT  1.000000              WT
   Variant   fitness updated_variant
0      10K  0.959845            D10K
1     643N  3.909999           E643N
2     370V  2.600643           N370V
3       5I 

In [10]:
iterations_four, labels_four = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_four
embeddings_pd = embeddings
labels_pd = labels_four
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [11]:
df_all
df_all.to_csv('t7/round4_all_new.csv', index=False)

In [12]:
df_test

,variant,y_pred,y_actual
8138,V429G,6.356751,NaN
8051,G424R,5.669336,NaN
7921,P417V,5.563483,NaN
15452,F814G,5.505366,NaN
8045,G424K,5.436164,NaN
...,...,...,...
11151,I587V,0.904100,NaN
14513,N764T,0.897736,NaN
4349,L229V,0.897673,NaN
4561,S241A,0.897293,NaN


In [13]:
df_test.to_csv('t7/round4_predictions_new.csv', index=False)

## Fanzor round 1

In [4]:
# import brenan data
dataset_name = 'fanzor_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/fanzor/'
file_type = 'csvs'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, experimental=experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [5]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
D289I,0.076350,-0.096517,-0.039485,0.022141,-0.140778,0.060272,-0.157692,0.053028,-0.101141,-0.058941,...,-0.163110,-0.056517,0.018743,-0.116311,0.010050,-0.082736,-0.068338,0.026717,-0.135575,0.090331
K54A,0.081564,-0.108703,-0.035292,0.022103,-0.143718,0.055708,-0.158636,0.051587,-0.099935,-0.056645,...,-0.159871,-0.058828,0.010209,-0.116926,0.010969,-0.086742,-0.072210,0.026919,-0.131113,0.088953
Y403W,0.083564,-0.103152,-0.035773,0.019590,-0.140105,0.058891,-0.161704,0.049788,-0.096187,-0.057417,...,-0.163753,-0.054475,0.013977,-0.118503,0.010152,-0.083362,-0.071264,0.024228,-0.136338,0.091752
N441W,0.085060,-0.102434,-0.034094,0.019113,-0.140549,0.057376,-0.160521,0.053643,-0.102079,-0.058173,...,-0.163639,-0.054493,0.015502,-0.117870,0.010070,-0.085296,-0.072524,0.028138,-0.135484,0.088637
E336I,0.081061,-0.103307,-0.037303,0.019810,-0.141395,0.060582,-0.161910,0.051436,-0.100989,-0.056924,...,-0.159600,-0.057073,0.012353,-0.119047,0.011846,-0.084845,-0.072005,0.029802,-0.134100,0.089094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D132W,0.079420,-0.105425,-0.030861,0.021674,-0.141255,0.057854,-0.160459,0.051246,-0.100951,-0.055793,...,-0.163258,-0.056315,0.014693,-0.118476,0.013192,-0.081750,-0.067947,0.028840,-0.137237,0.088986
R263G,0.083450,-0.106207,-0.033075,0.021937,-0.141942,0.058554,-0.160312,0.051613,-0.098827,-0.060687,...,-0.162723,-0.054210,0.014769,-0.119330,0.010308,-0.084599,-0.070771,0.026594,-0.133062,0.091404
V123P,0.082387,-0.104821,-0.032308,0.020482,-0.139570,0.055889,-0.161555,0.051725,-0.097712,-0.054436,...,-0.163546,-0.057811,0.011002,-0.118673,0.012264,-0.085351,-0.068471,0.026130,-0.137345,0.090820
I355T,0.082915,-0.105555,-0.036674,0.019969,-0.141193,0.058157,-0.162853,0.050937,-0.098292,-0.057070,...,-0.162178,-0.056842,0.014052,-0.117169,0.009596,-0.084980,-0.070661,0.026135,-0.134839,0.087683


In [6]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/fanzor/'
round_file_name = 'fanzor_Round1.xlsx'
fanzor_sequence = 'MKRKREDLTLWDAANVHKHKSMWYWWEYIRRKDMVNHEKTDCDVIQLLQSASVKKQKTQSDKFLTSFSVGIRPTKHQKRVLNEMLRVSNYTYNWCLWLVNEKGLKPHQFELQKIVCKTNANDVDPQYRMENDDWFFNNKMTSVKLTSCKNFCTSYKSAKSLKSKLKRPMSVSNIIQGSFCVPKLFIRHLSSKDVSTDNTNMQNRYICMMPDNFEKRSNPKERFLKLAKPITKIPPIDHDVKIVKRADGMFIMNIPCDPKYTRRNASNDTIEKRVCGIDPGGRTFATVYDPIDCCVFQVGIKEDKQYVISKLHNKIDHAHMHLTKAQNKKQQQAARERIVSLKKTHLKLKTFVDDIHLKLSSHLVKEYQYVALGKINVAQLVKTDRPKPLSKRAKRDLLYWQHYRFRQRLTHRTTNTECILDVQNEAYTSKTCGVCGTINKNLEKSETFYCDQCKYNTHRDVNGARNILLKSLRMFPFEKQQQ'
experimental_data = read_experimental_data(base_path, round_file_name, fanzor_sequence)
print(experimental_data)
df_list = [experimental_data]

  Variant   fitness updated_variant
0     27T  0.579502            E27T
1     20M  0.804204            K20M
2     22V  0.867465            M22V
3     46E  0.790334            Q46E
4     58Q  0.812868            T58Q
5     69M  0.434715            V69M
6     93A  0.976318            N93A
7    109N  0.428318           F109N
8     78T  0.568579            K78T
9      WT  1.000000              WT


In [7]:
iterations_one, labels_one = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [8]:
df_all
df_all.to_csv('fanzor/round1_all_new.csv', index=False)

In [9]:
df_test

,variant,y_pred,y_actual
7463,T199E,0.836856,NaN
1286,H17E,0.835102,NaN
2006,Q176E,0.833198,NaN
163,M320L,0.829511,NaN
2357,T196S,0.829295,NaN
...,...,...,...
435,F109S,0.602243,NaN
2454,F109A,0.592755,NaN
2870,F109T,0.575805,NaN
7667,F109H,0.569451,NaN


In [10]:
# write the dataframe to a csv file
df_test.to_csv('fanzor/round1_predictions_new.csv', index=False)

## Fanzor round 2

In [12]:
# import brenan data
dataset_name = 'fanzor_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/fanzor/'
file_type = 'csvs'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, experimental=experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [13]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
D289I,0.076350,-0.096517,-0.039485,0.022141,-0.140778,0.060272,-0.157692,0.053028,-0.101141,-0.058941,...,-0.163110,-0.056517,0.018743,-0.116311,0.010050,-0.082736,-0.068338,0.026717,-0.135575,0.090331
K54A,0.081564,-0.108703,-0.035292,0.022103,-0.143718,0.055708,-0.158636,0.051587,-0.099935,-0.056645,...,-0.159871,-0.058828,0.010209,-0.116926,0.010969,-0.086742,-0.072210,0.026919,-0.131113,0.088953
Y403W,0.083564,-0.103152,-0.035773,0.019590,-0.140105,0.058891,-0.161704,0.049788,-0.096187,-0.057417,...,-0.163753,-0.054475,0.013977,-0.118503,0.010152,-0.083362,-0.071264,0.024228,-0.136338,0.091752
N441W,0.085060,-0.102434,-0.034094,0.019113,-0.140549,0.057376,-0.160521,0.053643,-0.102079,-0.058173,...,-0.163639,-0.054493,0.015502,-0.117870,0.010070,-0.085296,-0.072524,0.028138,-0.135484,0.088637
E336I,0.081061,-0.103307,-0.037303,0.019810,-0.141395,0.060582,-0.161910,0.051436,-0.100989,-0.056924,...,-0.159600,-0.057073,0.012353,-0.119047,0.011846,-0.084845,-0.072005,0.029802,-0.134100,0.089094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D132W,0.079420,-0.105425,-0.030861,0.021674,-0.141255,0.057854,-0.160459,0.051246,-0.100951,-0.055793,...,-0.163258,-0.056315,0.014693,-0.118476,0.013192,-0.081750,-0.067947,0.028840,-0.137237,0.088986
R263G,0.083450,-0.106207,-0.033075,0.021937,-0.141942,0.058554,-0.160312,0.051613,-0.098827,-0.060687,...,-0.162723,-0.054210,0.014769,-0.119330,0.010308,-0.084599,-0.070771,0.026594,-0.133062,0.091404
V123P,0.082387,-0.104821,-0.032308,0.020482,-0.139570,0.055889,-0.161555,0.051725,-0.097712,-0.054436,...,-0.163546,-0.057811,0.011002,-0.118673,0.012264,-0.085351,-0.068471,0.026130,-0.137345,0.090820
I355T,0.082915,-0.105555,-0.036674,0.019969,-0.141193,0.058157,-0.162853,0.050937,-0.098292,-0.057070,...,-0.162178,-0.056842,0.014052,-0.117169,0.009596,-0.084980,-0.070661,0.026135,-0.134839,0.087683


In [14]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/fanzor/'
round_file_name_1 = 'fanzor_Round1.xlsx'
round_file_name_2 = 'fanzor_Round2.xlsx'
fanzor_sequence = 'MKRKREDLTLWDAANVHKHKSMWYWWEYIRRKDMVNHEKTDCDVIQLLQSASVKKQKTQSDKFLTSFSVGIRPTKHQKRVLNEMLRVSNYTYNWCLWLVNEKGLKPHQFELQKIVCKTNANDVDPQYRMENDDWFFNNKMTSVKLTSCKNFCTSYKSAKSLKSKLKRPMSVSNIIQGSFCVPKLFIRHLSSKDVSTDNTNMQNRYICMMPDNFEKRSNPKERFLKLAKPITKIPPIDHDVKIVKRADGMFIMNIPCDPKYTRRNASNDTIEKRVCGIDPGGRTFATVYDPIDCCVFQVGIKEDKQYVISKLHNKIDHAHMHLTKAQNKKQQQAARERIVSLKKTHLKLKTFVDDIHLKLSSHLVKEYQYVALGKINVAQLVKTDRPKPLSKRAKRDLLYWQHYRFRQRLTHRTTNTECILDVQNEAYTSKTCGVCGTINKNLEKSETFYCDQCKYNTHRDVNGARNILLKSLRMFPFEKQQQ'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, fanzor_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, fanzor_sequence)
print(experimental_data_1)
print(experimental_data_2)
df_list = [experimental_data_1, experimental_data_2]

  Variant   fitness updated_variant
0     27T  0.579502            E27T
1     20M  0.804204            K20M
2     22V  0.867465            M22V
3     46E  0.790334            Q46E
4     58Q  0.812868            T58Q
5     69M  0.434715            V69M
6     93A  0.976318            N93A
7    109N  0.428318           F109N
8     78T  0.568579            K78T
9      WT  1.000000              WT
   Variant   fitness updated_variant
0     214D  0.997700           E214D
1      70C  0.935783            G70C
2     131H  0.397414           N131H
3     238F  0.713195           H238F
4     223H  0.742610           F223H
5     128L  0.009221           R128L
6     239V  0.737319           D239V
7      40D  0.547851            T40D
8     282Y  0.805207           R282Y
9     466K  0.037661           N466K
10    379A  0.039456           Q379A
11    440N  1.164566           K440N
12      WT  1.000010              WT


In [15]:
iterations_two, labels_two = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_two
embeddings_pd = embeddings
labels_pd = labels_two
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [16]:
df_all
df_all.to_csv('fanzor/round2_all_new.csv', index=False)

In [17]:
df_test

,variant,y_pred,y_actual
97,T231N,0.875245,NaN
8939,K192N,0.873594,NaN
503,M129I,0.861280,NaN
4714,K192A,0.858800,NaN
8812,S191D,0.855105,NaN
...,...,...,...
3807,F109E,0.421076,NaN
8236,Q401L,0.406626,NaN
121,D278Y,0.403477,NaN
1112,D278N,0.401153,NaN


In [18]:
# write the dataframe to a csv file
df_test.to_csv('fanzor/round2_predictions_new.csv', index=False)

## Fanzor round 3

In [4]:
# import brenan data
dataset_name = 'fanzor_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/fanzor/'
file_type = 'csvs'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, experimental=experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [5]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
D289I,0.076350,-0.096517,-0.039485,0.022141,-0.140778,0.060272,-0.157692,0.053028,-0.101141,-0.058941,...,-0.163110,-0.056517,0.018743,-0.116311,0.010050,-0.082736,-0.068338,0.026717,-0.135575,0.090331
K54A,0.081564,-0.108703,-0.035292,0.022103,-0.143718,0.055708,-0.158636,0.051587,-0.099935,-0.056645,...,-0.159871,-0.058828,0.010209,-0.116926,0.010969,-0.086742,-0.072210,0.026919,-0.131113,0.088953
Y403W,0.083564,-0.103152,-0.035773,0.019590,-0.140105,0.058891,-0.161704,0.049788,-0.096187,-0.057417,...,-0.163753,-0.054475,0.013977,-0.118503,0.010152,-0.083362,-0.071264,0.024228,-0.136338,0.091752
N441W,0.085060,-0.102434,-0.034094,0.019113,-0.140549,0.057376,-0.160521,0.053643,-0.102079,-0.058173,...,-0.163639,-0.054493,0.015502,-0.117870,0.010070,-0.085296,-0.072524,0.028138,-0.135484,0.088637
E336I,0.081061,-0.103307,-0.037303,0.019810,-0.141395,0.060582,-0.161910,0.051436,-0.100989,-0.056924,...,-0.159600,-0.057073,0.012353,-0.119047,0.011846,-0.084845,-0.072005,0.029802,-0.134100,0.089094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
D132W,0.079420,-0.105425,-0.030861,0.021674,-0.141255,0.057854,-0.160459,0.051246,-0.100951,-0.055793,...,-0.163258,-0.056315,0.014693,-0.118476,0.013192,-0.081750,-0.067947,0.028840,-0.137237,0.088986
R263G,0.083450,-0.106207,-0.033075,0.021937,-0.141942,0.058554,-0.160312,0.051613,-0.098827,-0.060687,...,-0.162723,-0.054210,0.014769,-0.119330,0.010308,-0.084599,-0.070771,0.026594,-0.133062,0.091404
V123P,0.082387,-0.104821,-0.032308,0.020482,-0.139570,0.055889,-0.161555,0.051725,-0.097712,-0.054436,...,-0.163546,-0.057811,0.011002,-0.118673,0.012264,-0.085351,-0.068471,0.026130,-0.137345,0.090820
I355T,0.082915,-0.105555,-0.036674,0.019969,-0.141193,0.058157,-0.162853,0.050937,-0.098292,-0.057070,...,-0.162178,-0.056842,0.014052,-0.117169,0.009596,-0.084980,-0.070661,0.026135,-0.134839,0.087683


In [6]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/fanzor/'
round_file_name_1 = 'fanzor_Round1.xlsx'
round_file_name_2 = 'fanzor_Round2.xlsx'
round_file_name_3 = 'fanzor_Round3.xlsx'
fanzor_sequence = 'MKRKREDLTLWDAANVHKHKSMWYWWEYIRRKDMVNHEKTDCDVIQLLQSASVKKQKTQSDKFLTSFSVGIRPTKHQKRVLNEMLRVSNYTYNWCLWLVNEKGLKPHQFELQKIVCKTNANDVDPQYRMENDDWFFNNKMTSVKLTSCKNFCTSYKSAKSLKSKLKRPMSVSNIIQGSFCVPKLFIRHLSSKDVSTDNTNMQNRYICMMPDNFEKRSNPKERFLKLAKPITKIPPIDHDVKIVKRADGMFIMNIPCDPKYTRRNASNDTIEKRVCGIDPGGRTFATVYDPIDCCVFQVGIKEDKQYVISKLHNKIDHAHMHLTKAQNKKQQQAARERIVSLKKTHLKLKTFVDDIHLKLSSHLVKEYQYVALGKINVAQLVKTDRPKPLSKRAKRDLLYWQHYRFRQRLTHRTTNTECILDVQNEAYTSKTCGVCGTINKNLEKSETFYCDQCKYNTHRDVNGARNILLKSLRMFPFEKQQQ'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, fanzor_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, fanzor_sequence)
experimental_data_3 = read_experimental_data(base_path, round_file_name_3, fanzor_sequence)
print(experimental_data_1)
print(experimental_data_2)
print(experimental_data_3)
df_list = [experimental_data_1, experimental_data_2, experimental_data_3]

  Variant   fitness updated_variant
0     27T  0.579502            E27T
1     20M  0.804204            K20M
2     22V  0.867465            M22V
3     46E  0.790334            Q46E
4     58Q  0.812868            T58Q
5     69M  0.434715            V69M
6     93A  0.976318            N93A
7    109N  0.428318           F109N
8     78T  0.568579            K78T
9      WT  1.000000              WT
   Variant   fitness updated_variant
0     214D  0.997700           E214D
1      70C  0.935783            G70C
2     131H  0.397414           N131H
3     238F  0.713195           H238F
4     223H  0.742610           F223H
5     128L  0.009221           R128L
6     239V  0.737319           D239V
7      40D  0.547851            T40D
8     282Y  0.805207           R282Y
9     466K  0.037661           N466K
10    379A  0.039456           Q379A
11    440N  1.164566           K440N
12      WT  1.000010              WT
   Variant   fitness updated_variant
0     231N  0.718384           T231N
1     192N  

In [8]:
iterations_three, labels_three = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_three
embeddings_pd = embeddings
labels_pd = labels_three
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [9]:
df_all
df_all.to_csv('fanzor/round3_all_new.csv', index=False)

In [10]:
df_test

,variant,y_pred,y_actual
2008,K192T,1.031431,NaN
3469,K329Q,0.984432,NaN
3945,K440S,0.976134,NaN
1687,K220S,0.962656,NaN
415,K220A,0.960266,NaN
...,...,...,...
1170,D278G,0.421623,NaN
1112,D278N,0.413717,NaN
2454,F109A,0.411215,NaN
3807,F109E,0.409067,NaN


In [11]:
# write the dataframe to a csv file
df_test.to_csv('fanzor/round3_predictions_new.csv', index=False)

## Bxb1 round 1 (pretrained)

In [42]:
# import brenan data
dataset_name = 'bxb1_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [43]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
P52L,0.155984,-0.116075,0.148573,-0.028123,-0.128543,-0.084058,-0.072859,-0.046838,-0.002415,-0.005118,...,0.000492,0.028932,0.044435,-0.114348,-0.081508,-0.014825,-0.013309,0.009477,0.016132,-0.005828
R461Y,0.156687,-0.112199,0.145999,-0.024424,-0.130263,-0.083671,-0.071455,-0.046674,-0.005362,-0.004633,...,0.001077,0.030514,0.043889,-0.111547,-0.080573,-0.015240,-0.016286,0.011181,0.015980,-0.006640
R85A,0.154052,-0.115900,0.144834,-0.026544,-0.129637,-0.081927,-0.072439,-0.048122,-0.004115,-0.006136,...,0.003807,0.027205,0.042758,-0.113761,-0.078599,-0.019909,-0.019584,0.008901,0.016665,-0.006007
A315M,0.156422,-0.111366,0.145899,-0.025250,-0.130326,-0.082279,-0.071448,-0.045753,-0.003816,-0.005938,...,0.000151,0.029420,0.046772,-0.111136,-0.079974,-0.015162,-0.016042,0.010557,0.016900,-0.007189
R79C,0.154901,-0.110541,0.142594,-0.029071,-0.130484,-0.079025,-0.078174,-0.045398,-0.005141,-0.009236,...,-0.001456,0.027167,0.048554,-0.110430,-0.082701,-0.014656,-0.014808,0.012512,0.019153,-0.008661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P290D,0.156653,-0.111046,0.146798,-0.025523,-0.129217,-0.082911,-0.070534,-0.045017,-0.003130,-0.007678,...,0.000111,0.029905,0.048076,-0.112167,-0.079270,-0.014585,-0.017037,0.011904,0.019825,-0.004860
R487F,0.159038,-0.109513,0.146208,-0.024718,-0.131947,-0.081873,-0.071920,-0.047177,-0.005338,-0.006448,...,0.000096,0.029832,0.045403,-0.111501,-0.081506,-0.016406,-0.016590,0.010052,0.017690,-0.006641
K102M,0.157048,-0.113739,0.144254,-0.024531,-0.130623,-0.082974,-0.068666,-0.047273,-0.005326,-0.005729,...,0.001046,0.027864,0.045563,-0.111530,-0.080371,-0.017716,-0.016377,0.008898,0.018480,-0.007759
F314G,0.157193,-0.112016,0.145149,-0.025453,-0.131428,-0.082888,-0.072090,-0.047332,-0.003777,-0.005369,...,-0.000387,0.030322,0.044799,-0.111127,-0.079471,-0.016143,-0.015138,0.009161,0.018830,-0.006157


In [44]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
round_file_name_1 = 'bxb1_Round1_pretrained.xlsx'
bxb1_sequence = 'MRALVVIRLSRVTDATTSPERQLESCQQLCAQRGWDVVGVAEDLDVSGAVDPFDRKRRPNLARWLAFEEQPFDVIVAYRVDRLTRSIRHLQQLVHWAEDHKKLVVSATEAHFDTTTPFAAVVIALMGTVAQMELEAIKERNRSAAHFNIRAGKYRGSLPPWGYLPTRVDGEWRLVPDPVQRERILEVYHRVVDNHEPLHLVAHDLNRRGVLSPKDYFAQLQGREPQGREWSATALKRSMISEAMLGYATLNGKTVRDDDGAPLVRAEPILTREQLEALRAELVKTSRAKPAVSTPSLLLRVLFCAVCGEPAYKFAGGGRKHPRYRCRSMGFPKHCGNGTVAMAEWDAFCEEQVLDLLGDAERLEKVWVAGSDSAVELAEVNAELVDLTSLIGSPAYRAGSPQREALDARIAALAARQEELEGLEARPSGWEWRETGQRFGDWWREQDTAAKNTWLRSMNVRLTFDVRGGLTRTIDFGDLQEYEQHLRLGSVVERLHTGMS'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, bxb1_sequence)
print(experimental_data_1)
df_list = [experimental_data_1]

   Variant  fitness_raw Variant_real   fitness updated_variant
0      12K    35.217163         V12K  1.750215            V12K
1      14K    31.010153         D14K  1.541136            D14K
2      17K     2.670498         T17K  0.132718            T17K
3      45K    24.140190         D45K  1.199714            D45K
4      47K     0.040382         S47K  0.002007            S47K
5      49K    24.730773         A49K  1.229064            A49K
6     134K     0.072676        L134K  0.003612           L134K
7     137K     0.093551        I137K  0.004649           I137K
8     144K     9.514925        A144K  0.472871           A144K
9     145K     0.791633        A145K  0.039342           A145K
10    149K     0.089444        I149K  0.004445           I149K
11    199K     5.294259        H199K  0.263113           H199K
12    221K    17.470383        Q221K  0.868239           Q221K
13    226K    29.136050        N226K  1.447997           Q226K
14    229K     0.865538        E229K  0.043015         

In [45]:
iterations_one, labels_one = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [46]:
df_all
df_all.to_csv('bxb1/pretrained_round1_all_new.csv', index=False)

In [47]:
df_test

,variant,y_pred,y_actual
4134,D14S,1.235718,NaN
5172,D14G,1.175557,NaN
4676,Q28R,1.171850,NaN
7250,A412R,1.156191,NaN
4007,E20R,1.152608,NaN
...,...,...,...
4472,A315K,0.298830,NaN
3643,I149Q,0.279652,NaN
6537,I149C,0.278023,NaN
1796,H199Q,0.275105,NaN


In [48]:
df_test.to_csv('bxb1/pretrained_round1_predictions_new.csv', index=False)

## Bxb1 round 2 (pretrained)

In [17]:
# import brenan data
dataset_name = 'bxb1_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [18]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
P52L,0.155984,-0.116075,0.148573,-0.028123,-0.128543,-0.084058,-0.072859,-0.046838,-0.002415,-0.005118,...,0.000492,0.028932,0.044435,-0.114348,-0.081508,-0.014825,-0.013309,0.009477,0.016132,-0.005828
R461Y,0.156687,-0.112199,0.145999,-0.024424,-0.130263,-0.083671,-0.071455,-0.046674,-0.005362,-0.004633,...,0.001077,0.030514,0.043889,-0.111547,-0.080573,-0.015240,-0.016286,0.011181,0.015980,-0.006640
R85A,0.154052,-0.115900,0.144834,-0.026544,-0.129637,-0.081927,-0.072439,-0.048122,-0.004115,-0.006136,...,0.003807,0.027205,0.042758,-0.113761,-0.078599,-0.019909,-0.019584,0.008901,0.016665,-0.006007
A315M,0.156422,-0.111366,0.145899,-0.025250,-0.130326,-0.082279,-0.071448,-0.045753,-0.003816,-0.005938,...,0.000151,0.029420,0.046772,-0.111136,-0.079974,-0.015162,-0.016042,0.010557,0.016900,-0.007189
R79C,0.154901,-0.110541,0.142594,-0.029071,-0.130484,-0.079025,-0.078174,-0.045398,-0.005141,-0.009236,...,-0.001456,0.027167,0.048554,-0.110430,-0.082701,-0.014656,-0.014808,0.012512,0.019153,-0.008661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P290D,0.156653,-0.111046,0.146798,-0.025523,-0.129217,-0.082911,-0.070534,-0.045017,-0.003130,-0.007678,...,0.000111,0.029905,0.048076,-0.112167,-0.079270,-0.014585,-0.017037,0.011904,0.019825,-0.004860
R487F,0.159038,-0.109513,0.146208,-0.024718,-0.131947,-0.081873,-0.071920,-0.047177,-0.005338,-0.006448,...,0.000096,0.029832,0.045403,-0.111501,-0.081506,-0.016406,-0.016590,0.010052,0.017690,-0.006641
K102M,0.157048,-0.113739,0.144254,-0.024531,-0.130623,-0.082974,-0.068666,-0.047273,-0.005326,-0.005729,...,0.001046,0.027864,0.045563,-0.111530,-0.080371,-0.017716,-0.016377,0.008898,0.018480,-0.007759
F314G,0.157193,-0.112016,0.145149,-0.025453,-0.131428,-0.082888,-0.072090,-0.047332,-0.003777,-0.005369,...,-0.000387,0.030322,0.044799,-0.111127,-0.079471,-0.016143,-0.015138,0.009161,0.018830,-0.006157


In [19]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
round_file_name_1 = 'bxb1_Round1_pretrained.xlsx'
round_file_name_2 = 'bxb1_Round2_pretrained.xlsx'
bxb1_sequence = 'MRALVVIRLSRVTDATTSPERQLESCQQLCAQRGWDVVGVAEDLDVSGAVDPFDRKRRPNLARWLAFEEQPFDVIVAYRVDRLTRSIRHLQQLVHWAEDHKKLVVSATEAHFDTTTPFAAVVIALMGTVAQMELEAIKERNRSAAHFNIRAGKYRGSLPPWGYLPTRVDGEWRLVPDPVQRERILEVYHRVVDNHEPLHLVAHDLNRRGVLSPKDYFAQLQGREPQGREWSATALKRSMISEAMLGYATLNGKTVRDDDGAPLVRAEPILTREQLEALRAELVKTSRAKPAVSTPSLLLRVLFCAVCGEPAYKFAGGGRKHPRYRCRSMGFPKHCGNGTVAMAEWDAFCEEQVLDLLGDAERLEKVWVAGSDSAVELAEVNAELVDLTSLIGSPAYRAGSPQREALDARIAALAARQEELEGLEARPSGWEWRETGQRFGDWWREQDTAAKNTWLRSMNVRLTFDVRGGLTRTIDFGDLQEYEQHLRLGSVVERLHTGMS'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, bxb1_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, bxb1_sequence)
print(experimental_data_1)
print(experimental_data_2)
df_list = [experimental_data_1, experimental_data_2]

   Variant  fitness_raw Variant_real   fitness updated_variant
0      12K    35.217163         V12K  1.750215            V12K
1      14K    31.010153         D14K  1.541136            D14K
2      17K     2.670498         T17K  0.132718            T17K
3      45K    24.140190         D45K  1.199714            D45K
4      47K     0.040382         S47K  0.002007            S47K
5      49K    24.730773         A49K  1.229064            A49K
6     134K     0.072676        L134K  0.003612           L134K
7     137K     0.093551        I137K  0.004649           I137K
8     144K     9.514925        A144K  0.472871           A144K
9     145K     0.791633        A145K  0.039342           A145K
10    149K     0.089444        I149K  0.004445           I149K
11    199K     5.294259        H199K  0.263113           H199K
12    221K    17.470383        Q221K  0.868239           Q221K
13    226K    29.136050        N226K  1.447997           Q226K
14    229K     0.865538        E229K  0.043015         

In [20]:
iterations_two, labels_two = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_two
embeddings_pd = embeddings
labels_pd = labels_two
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [21]:
df_all
df_all.to_csv('bxb1/pretrained_round2_all_new.csv', index=False)

In [22]:
df_test

,variant,y_pred,y_actual
1636,D14A,1.112856,NaN
6556,D14Y,1.066313,NaN
3957,L9R,1.036923,NaN
3093,A360R,1.028954,NaN
4593,D14F,1.018732,NaN
...,...,...,...
2770,G152Q,0.313898,NaN
6537,I149C,0.297728,NaN
3643,I149Q,0.274097,NaN
6617,S47Q,0.273315,NaN


In [23]:
df_test.to_csv('bxb1/pretrained_round2_predictions_new.csv', index=False)

## Bxb1 round 3 (pretrained)

In [24]:
# import brenan data
dataset_name = 'bxb1_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)

In [25]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
P52L,0.155984,-0.116075,0.148573,-0.028123,-0.128543,-0.084058,-0.072859,-0.046838,-0.002415,-0.005118,...,0.000492,0.028932,0.044435,-0.114348,-0.081508,-0.014825,-0.013309,0.009477,0.016132,-0.005828
R461Y,0.156687,-0.112199,0.145999,-0.024424,-0.130263,-0.083671,-0.071455,-0.046674,-0.005362,-0.004633,...,0.001077,0.030514,0.043889,-0.111547,-0.080573,-0.015240,-0.016286,0.011181,0.015980,-0.006640
R85A,0.154052,-0.115900,0.144834,-0.026544,-0.129637,-0.081927,-0.072439,-0.048122,-0.004115,-0.006136,...,0.003807,0.027205,0.042758,-0.113761,-0.078599,-0.019909,-0.019584,0.008901,0.016665,-0.006007
A315M,0.156422,-0.111366,0.145899,-0.025250,-0.130326,-0.082279,-0.071448,-0.045753,-0.003816,-0.005938,...,0.000151,0.029420,0.046772,-0.111136,-0.079974,-0.015162,-0.016042,0.010557,0.016900,-0.007189
R79C,0.154901,-0.110541,0.142594,-0.029071,-0.130484,-0.079025,-0.078174,-0.045398,-0.005141,-0.009236,...,-0.001456,0.027167,0.048554,-0.110430,-0.082701,-0.014656,-0.014808,0.012512,0.019153,-0.008661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P290D,0.156653,-0.111046,0.146798,-0.025523,-0.129217,-0.082911,-0.070534,-0.045017,-0.003130,-0.007678,...,0.000111,0.029905,0.048076,-0.112167,-0.079270,-0.014585,-0.017037,0.011904,0.019825,-0.004860
R487F,0.159038,-0.109513,0.146208,-0.024718,-0.131947,-0.081873,-0.071920,-0.047177,-0.005338,-0.006448,...,0.000096,0.029832,0.045403,-0.111501,-0.081506,-0.016406,-0.016590,0.010052,0.017690,-0.006641
K102M,0.157048,-0.113739,0.144254,-0.024531,-0.130623,-0.082974,-0.068666,-0.047273,-0.005326,-0.005729,...,0.001046,0.027864,0.045563,-0.111530,-0.080371,-0.017716,-0.016377,0.008898,0.018480,-0.007759
F314G,0.157193,-0.112016,0.145149,-0.025453,-0.131428,-0.082888,-0.072090,-0.047332,-0.003777,-0.005369,...,-0.000387,0.030322,0.044799,-0.111127,-0.079471,-0.016143,-0.015138,0.009161,0.018830,-0.006157


In [26]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
round_file_name_1 = 'bxb1_Round1_pretrained.xlsx'
round_file_name_2 = 'bxb1_Round2_pretrained.xlsx'
round_file_name_3 = 'bxb1_Round3_pretrained.xlsx'
bxb1_sequence = 'MRALVVIRLSRVTDATTSPERQLESCQQLCAQRGWDVVGVAEDLDVSGAVDPFDRKRRPNLARWLAFEEQPFDVIVAYRVDRLTRSIRHLQQLVHWAEDHKKLVVSATEAHFDTTTPFAAVVIALMGTVAQMELEAIKERNRSAAHFNIRAGKYRGSLPPWGYLPTRVDGEWRLVPDPVQRERILEVYHRVVDNHEPLHLVAHDLNRRGVLSPKDYFAQLQGREPQGREWSATALKRSMISEAMLGYATLNGKTVRDDDGAPLVRAEPILTREQLEALRAELVKTSRAKPAVSTPSLLLRVLFCAVCGEPAYKFAGGGRKHPRYRCRSMGFPKHCGNGTVAMAEWDAFCEEQVLDLLGDAERLEKVWVAGSDSAVELAEVNAELVDLTSLIGSPAYRAGSPQREALDARIAALAARQEELEGLEARPSGWEWRETGQRFGDWWREQDTAAKNTWLRSMNVRLTFDVRGGLTRTIDFGDLQEYEQHLRLGSVVERLHTGMS'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, bxb1_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, bxb1_sequence)
experimental_data_3 = read_experimental_data(base_path, round_file_name_3, bxb1_sequence)
print(experimental_data_1)
print(experimental_data_2)
print(experimental_data_3)
df_list = [experimental_data_1, experimental_data_2, experimental_data_3]

   Variant  fitness_raw Variant_real   fitness updated_variant
0      12K    35.217163         V12K  1.750215            V12K
1      14K    31.010153         D14K  1.541136            D14K
2      17K     2.670498         T17K  0.132718            T17K
3      45K    24.140190         D45K  1.199714            D45K
4      47K     0.040382         S47K  0.002007            S47K
5      49K    24.730773         A49K  1.229064            A49K
6     134K     0.072676        L134K  0.003612           L134K
7     137K     0.093551        I137K  0.004649           I137K
8     144K     9.514925        A144K  0.472871           A144K
9     145K     0.791633        A145K  0.039342           A145K
10    149K     0.089444        I149K  0.004445           I149K
11    199K     5.294259        H199K  0.263113           H199K
12    221K    17.470383        Q221K  0.868239           Q221K
13    226K    29.136050        N226K  1.447997           Q226K
14    229K     0.865538        E229K  0.043015         

In [27]:
iterations_three, labels_three = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_three
embeddings_pd = embeddings
labels_pd = labels_three
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [28]:
df_all
df_all.to_csv('bxb1/pretrained_round3_all_new.csv', index=False)

In [29]:
df_test

,variant,y_pred,y_actual
3600,D14W,1.482967,NaN
8829,D14L,1.365908,NaN
4460,D14V,1.332520,NaN
9334,D14H,1.277728,NaN
6985,D14M,1.275672,NaN
...,...,...,...
4675,I149H,0.344950,NaN
8402,I137G,0.321267,NaN
6617,S47Q,0.315430,NaN
8032,I149N,0.310335,NaN


In [30]:
df_test.to_csv('bxb1/pretrained_round3_predictions_new.csv', index=False)

## Bxb1 round 1

In [39]:
# import brenan data
dataset_name = 'bxb1_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [40]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
P52L,0.155984,-0.116075,0.148573,-0.028123,-0.128543,-0.084058,-0.072859,-0.046838,-0.002415,-0.005118,...,0.000492,0.028932,0.044435,-0.114348,-0.081508,-0.014825,-0.013309,0.009477,0.016132,-0.005828
R461Y,0.156687,-0.112199,0.145999,-0.024424,-0.130263,-0.083671,-0.071455,-0.046674,-0.005362,-0.004633,...,0.001077,0.030514,0.043889,-0.111547,-0.080573,-0.015240,-0.016286,0.011181,0.015980,-0.006640
R85A,0.154052,-0.115900,0.144834,-0.026544,-0.129637,-0.081927,-0.072439,-0.048122,-0.004115,-0.006136,...,0.003807,0.027205,0.042758,-0.113761,-0.078599,-0.019909,-0.019584,0.008901,0.016665,-0.006007
A315M,0.156422,-0.111366,0.145899,-0.025250,-0.130326,-0.082279,-0.071448,-0.045753,-0.003816,-0.005938,...,0.000151,0.029420,0.046772,-0.111136,-0.079974,-0.015162,-0.016042,0.010557,0.016900,-0.007189
R79C,0.154901,-0.110541,0.142594,-0.029071,-0.130484,-0.079025,-0.078174,-0.045398,-0.005141,-0.009236,...,-0.001456,0.027167,0.048554,-0.110430,-0.082701,-0.014656,-0.014808,0.012512,0.019153,-0.008661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P290D,0.156653,-0.111046,0.146798,-0.025523,-0.129217,-0.082911,-0.070534,-0.045017,-0.003130,-0.007678,...,0.000111,0.029905,0.048076,-0.112167,-0.079270,-0.014585,-0.017037,0.011904,0.019825,-0.004860
R487F,0.159038,-0.109513,0.146208,-0.024718,-0.131947,-0.081873,-0.071920,-0.047177,-0.005338,-0.006448,...,0.000096,0.029832,0.045403,-0.111501,-0.081506,-0.016406,-0.016590,0.010052,0.017690,-0.006641
K102M,0.157048,-0.113739,0.144254,-0.024531,-0.130623,-0.082974,-0.068666,-0.047273,-0.005326,-0.005729,...,0.001046,0.027864,0.045563,-0.111530,-0.080371,-0.017716,-0.016377,0.008898,0.018480,-0.007759
F314G,0.157193,-0.112016,0.145149,-0.025453,-0.131428,-0.082888,-0.072090,-0.047332,-0.003777,-0.005369,...,-0.000387,0.030322,0.044799,-0.111127,-0.079471,-0.016143,-0.015138,0.009161,0.018830,-0.006157


In [41]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
round_file_name_1 = 'bxb1_Round1.xlsx'
bxb1_sequence = 'MRALVVIRLSRVTDATTSPERQLESCQQLCAQRGWDVVGVAEDLDVSGAVDPFDRKRRPNLARWLAFEEQPFDVIVAYRVDRLTRSIRHLQQLVHWAEDHKKLVVSATEAHFDTTTPFAAVVIALMGTVAQMELEAIKERNRSAAHFNIRAGKYRGSLPPWGYLPTRVDGEWRLVPDPVQRERILEVYHRVVDNHEPLHLVAHDLNRRGVLSPKDYFAQLQGREPQGREWSATALKRSMISEAMLGYATLNGKTVRDDDGAPLVRAEPILTREQLEALRAELVKTSRAKPAVSTPSLLLRVLFCAVCGEPAYKFAGGGRKHPRYRCRSMGFPKHCGNGTVAMAEWDAFCEEQVLDLLGDAERLEKVWVAGSDSAVELAEVNAELVDLTSLIGSPAYRAGSPQREALDARIAALAARQEELEGLEARPSGWEWRETGQRFGDWWREQDTAAKNTWLRSMNVRLTFDVRGGLTRTIDFGDLQEYEQHLRLGSVVERLHTGMS'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, bxb1_sequence)
print(experimental_data_1)
df_list = [experimental_data_1]

   Variant  fitness_raw   fitness updated_variant
0      23K     2.666789  0.483352            L23K
1      58T     0.000000  0.000000            R58T
2      79R     1.860380  0.337192            R79R
3     115K     3.122669  0.565980           T115K
4     141D     0.000000  0.000000           N141D
5     182M     1.822323  0.330294           E182M
6     230R     0.815088  0.147734           W230R
7     271N     0.000000  0.000000           T271N
8     318F     0.485051  0.087915           G318F
9     345H     1.172282  0.212475           W345H
10    376Y     0.906366  0.164278           E376Y
11    422P     1.026970  0.186137           G422P
12      WT     5.517278  1.000000              WT


In [42]:
iterations_one, labels_one = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [43]:
df_all
df_all.to_csv('bxb1/round1_all_new.csv', index=False)

In [44]:
df_test

,variant,y_pred,y_actual
8096,V375K,0.552319,NaN
5861,S25A,0.552317,NaN
2344,S238M,0.541832,NaN
4488,I123M,0.541639,NaN
5199,V375C,0.540567,NaN
...,...,...,...
9280,R58I,0.196812,NaN
3081,R58S,0.177974,NaN
2992,R58Q,0.159928,NaN
591,R58V,0.154377,NaN


In [45]:
df_test.to_csv('bxb1/round1_predictions_new.csv', index=False)

## Bxb1 round 2

In [46]:
# import brenan data
dataset_name = 'bxb1_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [47]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
P52L,0.155984,-0.116075,0.148573,-0.028123,-0.128543,-0.084058,-0.072859,-0.046838,-0.002415,-0.005118,...,0.000492,0.028932,0.044435,-0.114348,-0.081508,-0.014825,-0.013309,0.009477,0.016132,-0.005828
R461Y,0.156687,-0.112199,0.145999,-0.024424,-0.130263,-0.083671,-0.071455,-0.046674,-0.005362,-0.004633,...,0.001077,0.030514,0.043889,-0.111547,-0.080573,-0.015240,-0.016286,0.011181,0.015980,-0.006640
R85A,0.154052,-0.115900,0.144834,-0.026544,-0.129637,-0.081927,-0.072439,-0.048122,-0.004115,-0.006136,...,0.003807,0.027205,0.042758,-0.113761,-0.078599,-0.019909,-0.019584,0.008901,0.016665,-0.006007
A315M,0.156422,-0.111366,0.145899,-0.025250,-0.130326,-0.082279,-0.071448,-0.045753,-0.003816,-0.005938,...,0.000151,0.029420,0.046772,-0.111136,-0.079974,-0.015162,-0.016042,0.010557,0.016900,-0.007189
R79C,0.154901,-0.110541,0.142594,-0.029071,-0.130484,-0.079025,-0.078174,-0.045398,-0.005141,-0.009236,...,-0.001456,0.027167,0.048554,-0.110430,-0.082701,-0.014656,-0.014808,0.012512,0.019153,-0.008661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
P290D,0.156653,-0.111046,0.146798,-0.025523,-0.129217,-0.082911,-0.070534,-0.045017,-0.003130,-0.007678,...,0.000111,0.029905,0.048076,-0.112167,-0.079270,-0.014585,-0.017037,0.011904,0.019825,-0.004860
R487F,0.159038,-0.109513,0.146208,-0.024718,-0.131947,-0.081873,-0.071920,-0.047177,-0.005338,-0.006448,...,0.000096,0.029832,0.045403,-0.111501,-0.081506,-0.016406,-0.016590,0.010052,0.017690,-0.006641
K102M,0.157048,-0.113739,0.144254,-0.024531,-0.130623,-0.082974,-0.068666,-0.047273,-0.005326,-0.005729,...,0.001046,0.027864,0.045563,-0.111530,-0.080371,-0.017716,-0.016377,0.008898,0.018480,-0.007759
F314G,0.157193,-0.112016,0.145149,-0.025453,-0.131428,-0.082888,-0.072090,-0.047332,-0.003777,-0.005369,...,-0.000387,0.030322,0.044799,-0.111127,-0.079471,-0.016143,-0.015138,0.009161,0.018830,-0.006157


In [48]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/bxb1/'
round_file_name_1 = 'bxb1_Round1.xlsx'
round_file_name_2 = 'bxb1_Round2.xlsx'
bxb1_sequence = 'MRALVVIRLSRVTDATTSPERQLESCQQLCAQRGWDVVGVAEDLDVSGAVDPFDRKRRPNLARWLAFEEQPFDVIVAYRVDRLTRSIRHLQQLVHWAEDHKKLVVSATEAHFDTTTPFAAVVIALMGTVAQMELEAIKERNRSAAHFNIRAGKYRGSLPPWGYLPTRVDGEWRLVPDPVQRERILEVYHRVVDNHEPLHLVAHDLNRRGVLSPKDYFAQLQGREPQGREWSATALKRSMISEAMLGYATLNGKTVRDDDGAPLVRAEPILTREQLEALRAELVKTSRAKPAVSTPSLLLRVLFCAVCGEPAYKFAGGGRKHPRYRCRSMGFPKHCGNGTVAMAEWDAFCEEQVLDLLGDAERLEKVWVAGSDSAVELAEVNAELVDLTSLIGSPAYRAGSPQREALDARIAALAARQEELEGLEARPSGWEWRETGQRFGDWWREQDTAAKNTWLRSMNVRLTFDVRGGLTRTIDFGDLQEYEQHLRLGSVVERLHTGMS'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, bxb1_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, bxb1_sequence)
print(experimental_data_1, experimental_data_2)
df_list = [experimental_data_1, experimental_data_2]

   Variant  fitness_raw   fitness updated_variant
0      23K     2.666789  0.483352            L23K
1      58T     0.000000  0.000000            R58T
2      79R     1.860380  0.337192            R79R
3     115K     3.122669  0.565980           T115K
4     141D     0.000000  0.000000           N141D
5     182M     1.822323  0.330294           E182M
6     230R     0.815088  0.147734           W230R
7     271N     0.000000  0.000000           T271N
8     318F     0.485051  0.087915           G318F
9     345H     1.172282  0.212475           W345H
10    376Y     0.906366  0.164278           E376Y
11    422P     1.026970  0.186137           G422P
12      WT     5.517278  1.000000              WT    Variant  fitness_raw   fitness updated_variant
0     123M     0.016724  0.016723           I123M
1     166C     1.511531  1.511455           T166C
2     189Q     1.780421  1.780331           H189Q
3     245A     0.794822  0.794782           L245A
4      25A     1.411378  1.411307            S25A


In [49]:
iterations_two, labels_two = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_two
embeddings_pd = embeddings
labels_pd = labels_two
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [50]:
df_all
df_all.to_csv('bxb1/round2_all_new.csv', index=False)

In [51]:
df_test

,variant,y_pred,y_actual
4619,V375Q,1.250632,NaN
6300,V375W,1.239254,NaN
4733,V375A,1.183530,NaN
6332,T166Q,1.135125,NaN
5092,V375F,1.126244,NaN
...,...,...,...
6704,G318L,0.281939,NaN
8728,E267M,0.280385,NaN
9012,A405I,0.279286,NaN
3334,A412V,0.277518,NaN


In [52]:
df_test.to_csv('bxb1/round2_predictions_new.csv', index=False)

## CA round 1

In [64]:
# import brenan data
dataset_name = 'ca_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/ca/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})

In [65]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
D13F,0.085663,-0.070125,0.037689,0.019694,0.002905,-0.083943,-0.017667,0.060394,0.145624,-0.067333,...,-0.097398,-0.021168,-0.020748,-0.093693,-0.010967,0.057353,-0.071873,-0.056516,-0.071533,-0.130958
M45W,0.077280,-0.075316,0.040967,0.021198,0.007825,-0.089681,-0.018298,0.055321,0.142251,-0.056774,...,-0.091781,-0.016596,-0.017690,-0.089796,-0.006738,0.059956,-0.073939,-0.062558,-0.071477,-0.132031
A173E,0.087915,-0.072260,0.040313,0.022687,0.002946,-0.083454,-0.018720,0.055359,0.143977,-0.063539,...,-0.099815,-0.018082,-0.011997,-0.095527,-0.011362,0.054954,-0.070879,-0.061164,-0.070656,-0.126392
G47F,0.088420,-0.077352,0.038010,0.023851,0.006484,-0.085765,-0.022136,0.052470,0.145836,-0.062145,...,-0.098187,-0.017449,-0.015975,-0.092171,-0.010464,0.046052,-0.076304,-0.060668,-0.069655,-0.134855
M93T,0.069239,-0.085450,0.033526,0.028181,-0.001162,-0.094387,-0.023597,0.058478,0.140190,-0.059581,...,-0.080886,-0.020508,-0.030550,-0.093876,-0.003380,0.052741,-0.064846,-0.052142,-0.074726,-0.130032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
R170P,0.089024,-0.074984,0.043466,0.023683,0.002117,-0.085876,-0.012908,0.058917,0.142977,-0.063109,...,-0.092769,-0.017586,-0.019851,-0.097972,-0.008383,0.060735,-0.074547,-0.066768,-0.060328,-0.132468
L85P,0.091548,-0.075719,0.036663,0.019273,0.007470,-0.089777,-0.016545,0.057242,0.143495,-0.063106,...,-0.097158,-0.013028,-0.011680,-0.093433,-0.007895,0.054404,-0.078479,-0.058570,-0.063366,-0.135473
I56F,0.080827,-0.080963,0.041483,0.024030,0.011549,-0.092821,-0.023545,0.052908,0.136979,-0.068835,...,-0.091350,-0.013346,-0.022677,-0.098219,-0.003037,0.052579,-0.067524,-0.053649,-0.059933,-0.138120
L8G,0.079088,-0.077801,0.041376,0.020790,0.001375,-0.087771,-0.015298,0.057495,0.144516,-0.063691,...,-0.087798,-0.016448,-0.021661,-0.092828,-0.005738,0.057070,-0.075719,-0.053168,-0.061291,-0.126112


In [66]:
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/ca/'
round_file_name_1 = 'ca_Round1.xlsx'
ca_sequence = 'MTVTDDYLANNVDYASGFKGPLPMPPSKHIAIVACMDARLDVYRMLGIKEGEAHVIRNAGCVVTDDVIRSLAISQRLLGTREIILLHHTDCGMLTFTDDDFKRAIQDETGIRPTWSPESYPDAVEDVRQSLRRIEVNPFVTKHTSLRGFVFDVATGKLNEVTPAAALEARKEAELAAATAEQ'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, ca_sequence)
print(experimental_data_1)
df_list = [experimental_data_1]

   Variant   fitness updated_variant
0      13A  0.369211            D13A
1      35S  0.947054            C35S
2      26E  1.540771            P26E
3      22I  0.748429            L22I
4      15G  0.631504            A15G
5      20E  0.800809            G20E
6      25R  0.874041            P25R
7      30V  0.664379            I30V
8      40T  1.177277            L40T
9      55M  0.763146            V55M
10     65Y  0.711245            D65Y
11      WT  1.000000              WT


In [67]:
iterations_one, labels_one = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [68]:
df_all
df_all.to_csv('ca/round1_all_new.csv', index=False)

In [69]:
df_test

,variant,y_pred,y_actual
3253,P26D,1.068809,NaN
179,H87L,1.005771,NaN
1300,K49G,1.003750,NaN
312,K49S,0.989169,NaN
1431,K49C,0.989132,NaN
...,...,...,...
2680,D13T,0.759582,NaN
161,D65E,0.758149,NaN
3394,D107M,0.756664,NaN
3440,D13L,0.755272,NaN


In [70]:
df_test.to_csv('ca/round1_predictions_new.csv', index=False)

## MMFnuc Round 1

In [5]:
# import brenan data
dataset_name = 'mmfnuc_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/mmfnuc/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)

In [6]:
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
F251W,0.058680,-0.101877,-0.049105,0.043003,-0.129526,0.029136,-0.176315,0.014073,-0.099785,0.001786,...,-0.176302,-0.013019,0.040493,-0.126933,0.003337,-0.095834,-0.029323,0.022122,-0.108958,0.085466
S156P,0.057480,-0.102873,-0.054044,0.042988,-0.129897,0.027450,-0.177223,0.018766,-0.098628,0.000715,...,-0.173447,-0.020796,0.035839,-0.127225,0.005545,-0.095132,-0.033339,0.025640,-0.103198,0.087120
S180F,0.058544,-0.104004,-0.048321,0.038753,-0.127392,0.027275,-0.174910,0.015151,-0.099420,0.005351,...,-0.173295,-0.019856,0.035694,-0.125969,-0.002306,-0.092605,-0.033154,0.023564,-0.104630,0.087606
A419C,0.058057,-0.106263,-0.049922,0.041353,-0.129665,0.027440,-0.172086,0.018737,-0.098932,-0.000535,...,-0.174147,-0.017247,0.036708,-0.127276,0.001815,-0.096420,-0.030465,0.025425,-0.100106,0.087800
P299Y,0.057292,-0.106125,-0.052144,0.042111,-0.129207,0.026602,-0.171750,0.018487,-0.100901,-0.003204,...,-0.174812,-0.016793,0.037009,-0.124868,0.002369,-0.095742,-0.033622,0.024722,-0.098355,0.087069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F22H,0.058223,-0.102549,-0.053289,0.043090,-0.130043,0.030082,-0.174506,0.020153,-0.096832,-0.006312,...,-0.173114,-0.019959,0.037668,-0.126781,0.000861,-0.096356,-0.031056,0.025027,-0.103469,0.088209
S232P,0.054540,-0.103079,-0.051447,0.041830,-0.129405,0.027504,-0.173162,0.018984,-0.098990,-0.002792,...,-0.172950,-0.018743,0.036896,-0.127574,0.005526,-0.096237,-0.031903,0.023376,-0.107867,0.085076
K469Y,0.057801,-0.104625,-0.050990,0.041145,-0.129001,0.027085,-0.175023,0.018681,-0.098913,-0.000384,...,-0.170491,-0.017970,0.038323,-0.126963,0.002823,-0.097188,-0.031914,0.024631,-0.103980,0.084921
C431Y,0.058150,-0.104249,-0.050383,0.039418,-0.127600,0.026382,-0.177062,0.017804,-0.098297,-0.000005,...,-0.171520,-0.018806,0.039649,-0.128669,0.002352,-0.095632,-0.031668,0.026439,-0.103850,0.086796


In [7]:

base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/mmfnuc/'
round_file_name_1 = 'mmfnuc_Round1.xlsx'
mmfnuc_sequence = 'MKRKREQMTLWKAAFVNGQETFKSWIDKARMLELNCDVSSASSTHYSDLNLKTKCAKMEDKFMCTFSVGIRPTSKQKRTLNQMLKVSNHAYNWCNYLVKEKDFKPKQFDLQRVVTKTNSTDVPAEYRLPGDDWFFDNKMSSIKLTACKNFCTMYKSAQTNQKKTKVDLRNKDIAMLREGSFEVQKKYVRLLTEKDIPDERIRQSRIALMADNFSKSKKDWKERFLRLSKNVSKIPPLSHDMKVCKRPNGKFVLQIPCDPIYTRQIQVHTSDSICSIDPGGRTFATCYDPSNIKAFQIGPEADKKEVIHKYHEKIDYVHRLLAYAQKKKQTQAVQDRIGQLKKLHLKLKTYVDDVHLKLCSYLVKNYKLVVLGKISVSSIVRKDRPNHLAKKANRDLLCWQHYRFRQRLLHRVRGTDCEAIAQDERYTSKTCGNCGVKNNKLGGKETFICESCNYKTHRDVNGARNILCKYLGLFPFAA'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, mmfnuc_sequence)
print(experimental_data_1)
df_list = [experimental_data_1]

  Variant  fitness_raw   fitness updated_variant
0     35S     1.147238  1.147235            N35S
1     22I     1.310796  1.310793            F22I
2     25R     0.573165  0.573164            W25R
3     40T     0.818348  0.818347            S40T
4     50K     1.350812  1.350809            N50K
5     55M     0.908122  0.908120            C55M
6      WT     1.000002  1.000000              WT


In [8]:
iterations_one, labels_one = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [9]:
df_all
df_all.to_csv('mmfnuc/round1_all_new.csv', index=False)


In [10]:
df_test

,variant,y_pred,y_actual
2992,A14I,1.110493,NaN
3950,F22V,1.108385,NaN
1327,N50S,1.108227,NaN
2844,L49K,1.108054,NaN
1687,N50W,1.107230,NaN
...,...,...,...
789,W25G,0.859596,NaN
4074,W25Q,0.859500,NaN
8697,W25E,0.858582,NaN
2102,W25N,0.854021,NaN


In [11]:
df_test.to_csv('mmfnuc/round1_predictions_new.csv', index=False)

## MMFnuc Round 2

In [12]:
# import brenan data
dataset_name = 'mmfnuc_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/mmfnuc/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)

In [13]:
# replace WT Wild-type sequence index in embeddings with 'WT'
embeddings = embeddings.rename(index={'WT Wild-type sequence': 'WT'})
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
F251W,0.058680,-0.101877,-0.049105,0.043003,-0.129526,0.029136,-0.176315,0.014073,-0.099785,0.001786,...,-0.176302,-0.013019,0.040493,-0.126933,0.003337,-0.095834,-0.029323,0.022122,-0.108958,0.085466
S156P,0.057480,-0.102873,-0.054044,0.042988,-0.129897,0.027450,-0.177223,0.018766,-0.098628,0.000715,...,-0.173447,-0.020796,0.035839,-0.127225,0.005545,-0.095132,-0.033339,0.025640,-0.103198,0.087120
S180F,0.058544,-0.104004,-0.048321,0.038753,-0.127392,0.027275,-0.174910,0.015151,-0.099420,0.005351,...,-0.173295,-0.019856,0.035694,-0.125969,-0.002306,-0.092605,-0.033154,0.023564,-0.104630,0.087606
A419C,0.058057,-0.106263,-0.049922,0.041353,-0.129665,0.027440,-0.172086,0.018737,-0.098932,-0.000535,...,-0.174147,-0.017247,0.036708,-0.127276,0.001815,-0.096420,-0.030465,0.025425,-0.100106,0.087800
P299Y,0.057292,-0.106125,-0.052144,0.042111,-0.129207,0.026602,-0.171750,0.018487,-0.100901,-0.003204,...,-0.174812,-0.016793,0.037009,-0.124868,0.002369,-0.095742,-0.033622,0.024722,-0.098355,0.087069
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
F22H,0.058223,-0.102549,-0.053289,0.043090,-0.130043,0.030082,-0.174506,0.020153,-0.096832,-0.006312,...,-0.173114,-0.019959,0.037668,-0.126781,0.000861,-0.096356,-0.031056,0.025027,-0.103469,0.088209
S232P,0.054540,-0.103079,-0.051447,0.041830,-0.129405,0.027504,-0.173162,0.018984,-0.098990,-0.002792,...,-0.172950,-0.018743,0.036896,-0.127574,0.005526,-0.096237,-0.031903,0.023376,-0.107867,0.085076
K469Y,0.057801,-0.104625,-0.050990,0.041145,-0.129001,0.027085,-0.175023,0.018681,-0.098913,-0.000384,...,-0.170491,-0.017970,0.038323,-0.126963,0.002823,-0.097188,-0.031914,0.024631,-0.103980,0.084921
C431Y,0.058150,-0.104249,-0.050383,0.039418,-0.127600,0.026382,-0.177062,0.017804,-0.098297,-0.000005,...,-0.171520,-0.018806,0.039649,-0.128669,0.002352,-0.095632,-0.031668,0.026439,-0.103850,0.086796


In [14]:

base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/mmfnuc/'
round_file_name_1 = 'mmfnuc_Round1.xlsx'
round_file_name_2 = 'mmfnuc_Round2.xlsx'
mmfnuc_sequence = 'MKRKREQMTLWKAAFVNGQETFKSWIDKARMLELNCDVSSASSTHYSDLNLKTKCAKMEDKFMCTFSVGIRPTSKQKRTLNQMLKVSNHAYNWCNYLVKEKDFKPKQFDLQRVVTKTNSTDVPAEYRLPGDDWFFDNKMSSIKLTACKNFCTMYKSAQTNQKKTKVDLRNKDIAMLREGSFEVQKKYVRLLTEKDIPDERIRQSRIALMADNFSKSKKDWKERFLRLSKNVSKIPPLSHDMKVCKRPNGKFVLQIPCDPIYTRQIQVHTSDSICSIDPGGRTFATCYDPSNIKAFQIGPEADKKEVIHKYHEKIDYVHRLLAYAQKKKQTQAVQDRIGQLKKLHLKLKTYVDDVHLKLCSYLVKNYKLVVLGKISVSSIVRKDRPNHLAKKANRDLLCWQHYRFRQRLLHRVRGTDCEAIAQDERYTSKTCGNCGVKNNKLGGKETFICESCNYKTHRDVNGARNILCKYLGLFPFAA'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, mmfnuc_sequence)
experimental_data_2 = read_experimental_data(base_path, round_file_name_2, mmfnuc_sequence)
print(experimental_data_1, experimental_data_2)
df_list = [experimental_data_1, experimental_data_2]

  Variant  fitness_raw   fitness updated_variant
0     35S     1.147238  1.147235            N35S
1     22I     1.310796  1.310793            F22I
2     25R     0.573165  0.573164            W25R
3     40T     0.818348  0.818347            S40T
4     50K     1.350812  1.350809            N50K
5     55M     0.908122  0.908120            C55M
6      WT     1.000002  1.000000              WT    Variant  fitness_raw   fitness updated_variant
0      65Y     0.191288  0.191288            T65Y
1      15G     0.226811  0.226811            F15G
2      26E     0.438868  0.438868            I26E
3      14I     1.679332  1.679331            A14I
4      22V     1.636589  1.636589            F22V
5      50W     1.963885  1.963885            N50W
6      46I     1.105226  1.105225            Y46I
7      22L     1.140359  1.140358            F22L
8      49E     0.473967  0.473967            L49E
9      50A     1.025887  1.025887            N50A
10     50S     0.192993  0.192993            N50S
11      

In [15]:
iterations_two, labels_two = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_two
embeddings_pd = embeddings
labels_pd = labels_two
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [16]:
df_all
df_all.to_csv('mmfnuc/round2_all_new.csv', index=False)


In [17]:
df_test

,variant,y_pred,y_actual
8449,T53I,1.267518,NaN
8590,T53L,1.264053,NaN
3647,S43I,1.260009,NaN
6997,S42I,1.247397,NaN
3417,E33W,1.216781,NaN
...,...,...,...
4553,L34G,0.631872,NaN
3465,I26K,0.627114,NaN
878,L34N,0.620678,NaN
843,L34D,0.605848,NaN


In [18]:
df_test.to_csv('mmfnuc/round2_predictions_new.csv', index=False)

## R2

In [4]:
# import brenan data
dataset_name = 'r2_esm2_t48_15B_UR50D'
base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/r2/'
file_type = 'csvs'
embeddings_type = 'average'
experimental = True
embeddings = read_data(dataset_name, base_path, file_type, embeddings_type, experimental)

In [5]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,5110,5111,5112,5113,5114,5115,5116,5117,5118,5119
G91K,0.125095,-0.086747,0.007329,0.066044,-0.088842,-0.018201,-0.123120,0.048890,-0.063238,-0.024220,...,-0.061586,-0.006344,0.057469,-0.050334,-0.042349,-0.016509,-0.068606,0.012836,-0.129086,0.022432
N927I,0.122180,-0.083631,0.008685,0.064573,-0.088776,-0.018910,-0.124121,0.050447,-0.062081,-0.025686,...,-0.062493,-0.004403,0.056985,-0.052900,-0.042592,-0.014429,-0.069569,0.015393,-0.128585,0.021870
I824V,0.121158,-0.084260,0.007549,0.064432,-0.089038,-0.019331,-0.123460,0.049893,-0.061687,-0.025476,...,-0.062508,-0.004760,0.057204,-0.052061,-0.043694,-0.014540,-0.068647,0.014702,-0.128467,0.022632
I257T,0.121469,-0.084672,0.007914,0.064696,-0.088860,-0.020061,-0.125147,0.049035,-0.062149,-0.025158,...,-0.062353,-0.005594,0.056296,-0.052189,-0.044065,-0.014480,-0.068425,0.014220,-0.128542,0.020876
V56A,0.121873,-0.084203,0.008424,0.064599,-0.088758,-0.020137,-0.123685,0.050389,-0.061743,-0.025748,...,-0.062738,-0.005220,0.057436,-0.051883,-0.043081,-0.014980,-0.067994,0.014596,-0.128305,0.022357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
K1019H,0.121816,-0.084847,0.007153,0.065182,-0.088532,-0.019749,-0.123572,0.050393,-0.062215,-0.025013,...,-0.062319,-0.004719,0.056696,-0.052044,-0.043212,-0.014929,-0.069097,0.015089,-0.128408,0.022167
D812L,0.120452,-0.084344,0.007394,0.063507,-0.087286,-0.018784,-0.124111,0.047359,-0.061184,-0.024530,...,-0.063294,-0.005696,0.057122,-0.053182,-0.041089,-0.014853,-0.069275,0.015093,-0.128790,0.019908
F260L,0.121224,-0.084671,0.007690,0.064884,-0.089026,-0.019367,-0.124186,0.050523,-0.061800,-0.026249,...,-0.062903,-0.004626,0.057484,-0.052013,-0.042299,-0.014930,-0.067630,0.014060,-0.128773,0.022168
R135N,0.120981,-0.083602,0.005954,0.064894,-0.088813,-0.018985,-0.124834,0.050413,-0.061735,-0.025136,...,-0.063572,-0.004989,0.057836,-0.051759,-0.040987,-0.014490,-0.069512,0.014465,-0.127477,0.021782


In [6]:

base_path = '/Users/matteodibernardo/Documents/GitHub/directed_evolution/notebooks/r2/'
round_file_name_1 = 'r2_Round1.xlsx'
mmfunc_sequence = 'VKVTVPDKNPPCPCCSTRLNSVLALIDHLKGSHGKRRVCFRCAKCGRENFNHHSTVCHFAKCKGPSEEKPPVGEWICEVCGRDFTTKIGLGQHKRLAHPMVRNQERIDASQPKETSNRGAHKKCWTKEEEELLARLEVQFEGHKNINKLIAEHITTKTNKQISDKRRQMTRKDKGEGGAAGKLGPDTGRGNHSQAKVGNNGLGGNQLPGGPAATKDKAGCHLDKEEGNRIAISQQKKGRLQGRYHKEIKRRLEEGVINTFTKAFKQLLECQEVQPLINKTAQDCFGLLESACHIRTALRGKNKKETQEKPTGGQCLKWMKKRAVKKGNYLRFQRLFHLDRGKLARIILDDIECLSCDIAPSEIYSVFKARWETPGQFAGLGNFKSTGKADNKAFSDLITAKEIKKNVQEMSKGSAPGPDGIAIGDIKGMDPGYSRTAELFNLWLTSGEIPDMVRGCRTVLIPKSTQPERLKDINNWRPITIGSILLRLFSRIITARMTKACPLNPRQRGFIRAAGCSENLKLLQTIIRTAKSEHRPLGVVFVDIAKAFDTVSHQHILHVLQQRGVDPHIIGLVSNMYKDISTFVTTKKDTHTDKIQIRVGVKQGDPLSPLLFNLAMDPLLCKLEESGNGFHRGGHTITAMAFADDLVLLSDSWENMEKNIEILEAFCDLTGLKTQGQKCHGFYIKPTKDSYTVNNCAAWTIYGTPLNMINPGDSEKYLGLQIDPWTGIARSNISSKLDSWLERINQAPLKPLQKLDILKTYTIPRLTYMVDHSEMKAGALEALDLQIRSAVKDWLHLPSCTCDAILYVSTKDGGLGVTKLAGLIPSIQARRLHRIAQSPDETMKAFLDKEQMEKQYAKLWVQAGGKREKIPSIWDALPTPVLLTTSDTLSEWEAPNPKSKYPRPCNWRRKEFEKWTKLQCQGRGIQNFKGDVISNNWIQNYRRIPHRKLLTAVQLRANVYPTREFLGRGRGDDCVKFCRHCEVDLETCGHIISYCPVTKEARIKRHNRICERLIEEAEKKDWVVFKEPHIRDAVKELFKPDLIFVKEDRALVVDVTVRFEATTTSLEEAAIEKVDKYKRLETEVRSLTNAKDVLFMGFPLGARGKWYQGNFKLLDMLGLSESRQVTVAKTLSTDALISSVDIVHMFASKARKMNLVTV'
experimental_data_1 = read_experimental_data(base_path, round_file_name_1, mmfunc_sequence)
print(experimental_data_1)
df_list = [experimental_data_1]

  Variant   fitness updated_variant
0     23K  1.161144            L23K
1     79R  0.913773            V79R
2    115K  1.245699           T115K
3    141D  0.875473           E141D
4    271N  0.785357           Q271N
5    318F  1.262595           W318F
6    345H  0.820908           R345H
7    422P  1.115601           A422P
8      WT  1.000000              WT


In [7]:
iterations_one, labels_one = create_dataframes(df_list, embeddings.index)

iteration_old = iterations_one
embeddings_pd = embeddings
labels_pd = labels_one
measured_var = 'fitness'
regression_type = 'randomforest'
num_mutants_per_round = 16
final_round = 16

df_test, df_all = top_layer(
    iter_train=iteration_old['iteration'].unique().tolist(),
    iter_test=1001,
    embeddings_pd=embeddings_pd,
    labels_pd=labels_pd,
    measured_var=measured_var,
    regression_type=regression_type,
    top_n=None,
    final_round=final_round
)

Embeddings and labels are aligned


In [8]:
df_all
df_all.to_csv('r2/round1_all_new.csv', index=False)

In [9]:
df_test

,variant,y_pred,y_actual
11359,W318V,1.147273,NaN
1121,W318I,1.134502,NaN
15462,W318Y,1.132448,NaN
14588,W318L,1.132188,NaN
16561,W318S,1.126092,NaN
...,...,...,...
4346,K144V,0.935839,NaN
10158,E305H,0.935395,NaN
10704,Q307H,0.934809,NaN
13833,K531T,0.934760,NaN


In [10]:
df_test.to_csv('r2/round1_predictions_new.csv', index=False)